In [1]:
# you-get是一个可以用命令行下载网页视频的程序，只要运行下面这段代码安装即可
# !pip install you-get -i https://pypi.tuna.tsinghua.edu.cn/simple/

In [2]:
import os
import shutil
import time
import numpy as np
import warnings
from tqdm.notebook import tqdm

from PIL import Image, ImageOps, ImageFilter, ImageEnhance
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg
import matplotlib

# 防止弹出警告标志
warnings.filterwarnings("ignore")

In [3]:
def downloadMp4(url, downloadName, downloadPath):
    '''
    这是下载函数
    url表示网址
    downloadName是不包括文件后缀的文件名
    downloadPath是想要下载的地址
    '''
    os.system('you-get {} -O {} -o {} --playlist --no-caption -a'.format(
        url, downloadName, downloadPath))

In [4]:
def createFolder(path):
    '''
    path表示目标文件夹地址
    
    该函数将（创建并）重置该文件夹
    '''
    is_exists = os.path.exists(path)
    if not is_exists:
        os.makedirs(path)
        print('path of {} is build'.format(path))
    else:
        shutil.rmtree(path)
        os.makedirs(path)
        print('path of {} already exist and rebuild'.format(path))

In [5]:
def getImage(videoPath, imagePath, FPS=10, leftTime=0, rightTime=120):
    '''
    这是抽帧函数
    videoPath是视频地址
    imagePath是输出图像的文件夹
    FPS是视频的帧率
    leftTime是截取视频的左时间点
    rightTime是截取视频的右时间点
    
    '''
    # 建立存放输出图片的文件夹
    createFolder(imagePath)

    for img_count in tqdm(range(int(FPS * (rightTime - leftTime)))):
        cutTime = leftTime + img_count / FPS
        os.system('ffmpeg -i {} -f image2 -ss {} -vframes 1 {}.png'.format(
            videoPath, str(cutTime), imagePath + '/' + str(img_count)))

In [6]:
def img2Up(imgFile, blur=25, alpha=1.0, contrast=10):
    '''
    该函数将图片转换为素描并提高图像对比度
    imgFile是输入的图片地址
    blur是模糊参数
    alpha是透明度参数
    contrast是对比度参数
    
    该函数的输出是单通道素描对比度图片
    '''

    # 定义mixedConversion函数，可将两目标像素点按一定比例混合
    mixedConversion = lambda a, b, alpha: min(int(a * 255 /
                                                  (256 - b * alpha)), 255)

    # 转化成img对象，将文件转成单通道灰度图
    img1 = Image.open(imgFile).convert('L')

    # 对图像灰度值进行反转，反转后的灰度值为 255 - 原始灰度值
    img2 = img1.copy()
    img2 = ImageOps.invert(img2)

    # 将图像模糊化
    for i in range(blur):
        img2 = img2.filter(ImageFilter.BLUR)
        width, height = img1.size

    # 对单通道灰度图与模糊化后的图像进行混合
    for x in range(width):
        for y in range(height):
            a = img1.getpixel((x, y))
            b = img2.getpixel((x, y))
            img1.putpixel((x, y), mixedConversion(a, b, alpha))

    # 提高图像对比度操作
    contrastEnhance = ImageEnhance.Contrast(img1)
    imgUped = contrastEnhance.enhance(contrast)

    return imgUped

In [7]:
def toWhatChar(matrix, dotsMatrixSize=2):
    '''
    matrix表示转化的像素矩阵，允许输入1*1或2*2的
    
    dotsMatrixSize标识矩阵尺寸    
    '''
    if dotsMatrixSize == 2:
        return toWhatChar4(matrix)
    return '#' if matrix == 0 else ' '


def toWhatChar4(matrix):
    matrix = matrix == 0
    if (matrix == np.array([[1, 1], [1, 1]])).all():
        return '井'
    if (matrix == np.array([[1, 1], [1, 0]])).all():
        return '『'
    if (matrix == np.array([[1, 1], [0, 1]])).all():
        return '𠃍'
    if (matrix == np.array([[1, 1], [0, 0]])).all():
        return '冖'

    if (matrix == np.array([[1, 0], [1, 1]])).all():
        return '𠃊'
    if (matrix == np.array([[1, 0], [1, 0]])).all():
        return '纟'
    if (matrix == np.array([[1, 0], [0, 1]])).all():
        return '㇏'
    if (matrix == np.array([[1, 0], [0, 0]])).all():
        return '　'

    if (matrix == np.array([[0, 1], [1, 1]])).all():
        return '』'
    if (matrix == np.array([[0, 1], [1, 0]])).all():
        return '㇀'
    if (matrix == np.array([[0, 1], [0, 1]])).all():
        return '刂'
    if (matrix == np.array([[0, 1], [0, 0]])).all():
        return '　'

    if (matrix == np.array([[0, 0], [1, 1]])).all():
        return '灬'
    if (matrix == np.array([[0, 0], [1, 0]])).all():
        return '　'
    if (matrix == np.array([[0, 0], [0, 1]])).all():
        return '　'
    if (matrix == np.array([[0, 0], [0, 0]])).all():
        return '　'

In [8]:
def img2txt(imgFile, txtFile, method, figuresize=100000):
    '''
    这是我们本次作业的主要函数，后面的imgs2txt也即重复调用该函数
    
    imgFile是输入的图片地址
    txtFile是输出的文本的地址
    method表示我们选用的方法，1表示只使用#，2表示使用16种汉字字符，3表示使用半角字符映射灰度值
    figuresize表示目标的像素大小，之后会自动调整长宽
    '''
    
    if method == 1:
        dotsMatrixSize, ifHan = 1, True
    elif method == 2:
        dotsMatrixSize, ifHan = 2, True
    else:
        dotsMatrixSize, ifHan = 1, False
    
    # 用result存放转化后的txtFile文本
    result = ''
    
    
    # 存放一些使用方法3需要的参数
    chars = 'MNHQ$OC?&>!:-; '
    chars = chars[::-1]
    N = len(chars)
    step = 256 // N + 1

    # 这里将图片尺寸等比缩放到指定的像素大小
    orgWidth, orgHeight = Image.open(imgFile).convert('L').size
    charWidth = int(np.sqrt(orgWidth / orgHeight * figuresize))
    charHeight = int(np.sqrt(orgHeight / orgWidth * figuresize))

    # 这里调用img2Up函数，并用ANTIALIAS方法调整图片尺寸，最后得到一个图片数组
    imgArray = np.array(img2Up(imgFile).resize(
        (charWidth * dotsMatrixSize, charHeight * dotsMatrixSize),
        Image.ANTIALIAS))
    
    # 逐像素点进行转化
    for row in range(charHeight):
        for column in range(charWidth):
            
            if ifHan:
                # 方法1和方法2使用前面toWhatChar函数指定的转换法则
                result += toWhatChar(
                    imgArray[row * dotsMatrixSize:(row + 1) * dotsMatrixSize,
                             column * dotsMatrixSize:(column + 1) *
                             dotsMatrixSize], dotsMatrixSize)
            else:
                # 方法3即根据灰度值进行更细分区，转化对应字符
                result += chars[imgArray[row][column] // step]
        result += '\n'
    
    # 用file表示最后存放的文件
    file = open(txtFile, 'w')
    file.write(result)
    file.close()

In [9]:
def imgs2txt(imagePath, txtPath, method, figuresize=100000):
    '''
    这是一个将图片批量转成对应文本文件的函数
    
    imagePath是输入图像的文件夹（只能有符合前面抽帧命名的图片）
    txtPath是输出txt的文件夹
    
    method表示我们选用的方法，1表示只使用#，2表示使用16种汉字字符，3表示使用半角字符映射灰度值
    figuresize表示目标的像素大小，之后会自动调整长宽

    '''
    
    # 建立存放输出文本文件的文件夹
    createFolder(txtPath) 
    files = os.listdir(imagePath)   # 读入图片文件夹
    num_png = len(files)     # 统计文件夹中的文件个数

    # 批量使用img2txt进行转换
    for img_count in tqdm(range(num_png)):
        imgFile = imagePath + '/' + str(img_count) + '.png'
        txtFile = txtPath + '/' + str(img_count) + '.txt'
        img2txt(imgFile, txtFile, method, figuresize)

In [10]:
def play(txtPath, FPS=10):
    '''
    该函数将在命令行中批量输出txt文件。
    不过该函数将在play.py文件中使用，这里只做展示
    另外为了完整表现输出结果，您需要适当调整您的命令行窗口大小、行间距和字符间距
    
    在命令行执行play.py时，应当采用如下格式，间隔是空格
    “python play.py地址 txt文件夹 播放帧率”
    如
    “python play.py txtPath 10”
    '''
    files = os.listdir(txtPath)   # 读入文件夹
    num_png = len(files)
    
    # 使用cat方法打印txt文件
    for txt_count in range(num_png):
        os.system('cat ' + txtPath + '/' +str(txt_count) + '.txt')
        time.sleep(1.0/FPS)

In [11]:
# 配置参数
url = 'https://www.bilibili.com/video/BV1Js411t7fm'
downloadName = 'test'
downloadPath = os.getcwd()
downloadMp4(url, downloadName, downloadPath)
# 如果您想使用本地的文件，只需在这里指定文件地址即可并注释上面的四行代码
videoPath = downloadPath + '/' + downloadName + '.mp4'


FPS = 0.5
leftTime, rightTime = 0, 20  # 这里为了简便起见，这里抽取前20秒、每2秒一抽共10张图片
imagePath = 'imagePath'
figuresize = 100000

# 抽帧函数，如果已经有现成的图片文件夹，也可以注释掉下面一行代码
getImage(videoPath, imagePath, FPS, leftTime, rightTime)

# 分别调用三种转换方法
for method in range(1, 4):
    txtPath = 'txtPath' + str(method)
    imgs2txt(imagePath, txtPath, method, figuresize)

path of imagePath already exist and rebuild



path of txtPath1 already exist and rebuild



path of txtPath2 already exist and rebuild



path of txtPath3 already exist and rebuild
